# 00 - Ingesta y Optimización de Datos del CIC-IDS-2017

**Objetivo:** Este notebook implementa el pipeline de ingesta de datos. Lee los 8 archivos CSV originales del dataset, los une, realiza una limpieza fundamental y guarda el resultado en un único archivo Parquet optimizado para su uso en análisis posteriores.

In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 1. Carga y Unión de Archivos CSV

Se localizan todos los archivos `.csv` en el directorio de datos crudos y se concatenan en un solo DataFrame de Pandas.

In [2]:
# Definir la ruta a los datos crudos
path_to_csvs = r'../data/raw/DatasetCICIDS2017/MachineLearningCVE/'

# Encontrar todas las rutas de los archivos CSV
csv_files = glob.glob(os.path.join(path_to_csvs, "*.csv"))

# Cargar todos los archivos CSV en una lista de DataFrames usando una list comprehension
df_list = [pd.read_csv(f) for f in csv_files]

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_list, ignore_index = True)

# Imprimir las dimensiones finales como confirmación
print(f"Dataset combinado cargado. Dimensiones: {df.shape}")

Dataset combinado cargado. Dimensiones: (2830743, 79)


## 2. Limpieza y Optimización

Esta fase prepara el DataFrame para el análisis. Incluye los siguientes sub-pasos:
1.  **Limpieza de Nombres de Columnas:** Se eliminan los espacios en blanco.
2.  **Manejo de Valores Inválidos:** Se reemplazan los valores infinitos por `NaN` y se eliminan las filas correspondientes.
3.  **Limpieza de Etiquetas:** Se corrigen caracteres mal codificados en la columna `Label`.

In [3]:
# Aplicar la secuencia de limpieza de datos
df.columns = df.columns.str.strip()
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna(inplace = True)

# Limpiar la columna 'Label' encadenando los métodos de string
df['Label'] = (df['Label']
               .str.replace('�', ' ', regex = False)
               .str.replace(r'\s+', ' ', regex = True)
               .str.strip())

## 3. Optimización Inicial y Guardado en Formato Parquet

Finalmente, se aplica una función para reducir drásticamente el uso de memoria del DataFrame limpio, convirtiendo las columnas numéricas al tipo de dato más pequeño posible (`downcasting`) y las columnas de texto a un tipo `category` eficiente. El DataFrame optimizado se guarda en formato `Parquet`. Este formato es columnar, comprimido y mucho más rápido de leer que el `.csv`, lo que agilizará todos los análisis futuros.

In [4]:
def optimize_memory(df):
    """
    Itera sobre todas las columnas de un DataFrame y modifica los tipos de datos
    para reducir el uso de memoria, aplicando 'downcasting' a tipos numéricos
    y convirtiendo tipos 'object' a 'category'.

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif col_type == 'object':
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final: {end_mem:.2f} MB')
    print(f'Reducción del {100 * (start_mem - end_mem) / start_mem:.1f}%')
    return df

# Aplicar la función de optimización de memoria
df = optimize_memory(df)

# Guardar el DataFrame final en formato Parquet
path_to_save = '../data/processed/cic_ids_2017_optimized.parquet'
df.to_parquet(path_to_save)

print(f"\nDataFrame optimizado y guardado exitosamente en: {path_to_save}")

Uso de memoria inicial: 1726.00 MB
Uso de memoria final: 655.34 MB
Reducción del 62.0%

DataFrame optimizado y guardado exitosamente en: ../data/processed/cic_ids_2017_optimized.parquet
